In [1]:
import csv
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, SKOS, DCTERMS, DC
import uuid
#from django.core.serializers.json import DjangoJSONEncoder
from functools import reduce

NS = Namespace("")

In [2]:
file = 'rdm/source/rdm_new.csv'
#concept_uri;parent;label;comment;aat_identifier;;parent_tw
voc_name = 'Datahub'
voc_uri = 'https://data.antwerp.be/id/term/Datahub'

In [3]:
print(uuid.uuid5(uuid.NAMESPACE_DNS, 'bulle'))
print(uuid.uuid5(uuid.NAMESPACE_DNS, 'bulle'))

c12f5d50-1187-53ff-b5a5-e1102e904bfb
c12f5d50-1187-53ff-b5a5-e1102e904bfb


In [18]:
g = Graph()
g.bind("ns", NS)
g.bind("skos", SKOS)
g.bind("dcterms", DCTERMS)
g.bind("dc", DC)
    
with open(f'source/{file}', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    next(reader, None)  # skip the headers
    ns_voc_name = NS[voc_name]
    ns_voc_uri = NS[voc_uri]
    g.add((ns_voc_uri, RDF.type, SKOS['ConceptScheme']))
    g.add((ns_voc_uri, DCTERMS.title, Literal(voc_name,lang="nl")))
    for row in reader:
        #uri = NS[str(row[0])]
        uri = NS[f'http://localhost:8000/{uuid.uuid5(uuid.NAMESPACE_DNS, row[0])}']
        parent_uri = NS[f'http://localhost:8000/{uuid.uuid5(uuid.NAMESPACE_DNS, row[1])}']
        g.add((uri, RDF.type, SKOS.Concept))
        g.add((uri, SKOS.prefLabel, Literal(row[2], lang="nl")))
        g.add((uri, SKOS.inScheme, ns_voc_uri))
        uri_str = f'https://data.antwerp.qq/##€€id€€:§€€{str(uuid.uuid5(uuid.NAMESPACE_DNS, row[0]))}€€,§€€value€€:§€€{row[0]}**'            
        #g.add((uri, DCTERMS.identifier, NS[uri_str]))
        
        
        if str(row[1]) == 'https://data.antwerp.be/id/term/Datahub':
            g.add((ns_voc_uri, SKOS.hasTopConcept, uri))
        else:    
            g.add((parent_uri, SKOS.narrower, uri))

        if len(str(row[3])) > 0:
           g.add((uri, SKOS.editorialNote, Literal(row[3], lang="nl")))
        if len(str(row[4])) > 0:
            aat_uri_str = f'https://data.antwerp.qq/##€€id€€:§€€{str(uuid.uuid5(uuid.NAMESPACE_DNS, row[4]))}€€,§€€value€€:§€€{row[4]}**'            
            #g.add((uri, DCTERMS.identifier, NS[aat_uri_str]))

skos_data = g.serialize(format='pretty-xml').decode('utf-8')

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

d = {"https://data.antwerp.qq/": "", "##": "{", "**": "\"}", "€€": "\"", "\"{": "{", "}\"": "}", 
     "rdf:resource={":"{", "dcterms:identifier": "dcterms:identifier xml:lang=\"nl\"", "§": " ", "}/>": "}</dcterms:identifier>"}

skos_cleaned = replace_all(skos_data, d)

with open(f'out/{voc_name}_out_test.xml', "w") as f:
    f.write(skos_cleaned)
    
print(skos_cleaned)
skos_data = ''
del g

<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
  xmlns:dcterms="http://purl.org/dc/terms/"
  xmlns:skos="http://www.w3.org/2004/02/skos/core#"
  xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
>
  <skos:Concept rdf:about="http://localhost:8000/df5696b2-b301-5c1b-86ed-fe2b36c22fd0">
    <skos:narrower>
      <skos:Concept rdf:about="http://localhost:8000/089b8a1e-1903-524c-afa7-c06353306bfd">
        <skos:prefLabel xml:lang="nl">Niet gedefinieerd; conversies</skos:prefLabel>
        <skos:inScheme rdf:resource="https://data.antwerp.be/id/term/Datahub"/>
      </skos:Concept>
    </skos:narrower>
    <skos:narrower>
      <skos:Concept rdf:about="http://localhost:8000/a5e5e26b-ff43-59da-a5b4-d171a8b1c148">
        <skos:prefLabel xml:lang="nl">Vertaler</skos:prefLabel>
        <skos:inScheme rdf:resource="https://data.antwerp.be/id/term/Datahub"/>
      </skos:Concept>
    </skos:narrower>
    <skos:narrower>
      <skos:Concept rdf:about="http://localhost:8000/652170bd-9037-57